In [1]:
import numpy as np
import json
import itertools
from matplotlib import pyplot as plt
import pymatgen as mg
from pymatgen.ext.matproj import MPRester
from pprint import pprint
from ase.io import read
from ase.thermochemistry import IdealGasThermo
from py_box3 import saveplt, parse_formula
from py_box3 import constants as c
from py_box3.thermo.shomate import Shomate, Phase, generate_Cp_data
from py_box3.thermo.pymatgen import get_unit_energy

def get_metal(elements):
	"""
	Returns the metal element given a string of elements separted by hyphens.

	Parameters
		elements - string
			Elements separated by hyphens.
			e.g. 'Al-O'
	Returns
		string
			Metal element
			e.g. 'Al'
	"""
	return elements.replace('-O', '').replace('O-', '')

def get_oxygen_number(specie):
	"""
	Returns the number of oxygen found in the stoichiometric formula

	Parameters
		specie - dict
			Dictionary of the specie that should contain the key 'unit_cell_formula'
	Returns
		float
			Number of oxygen
	"""
	elements = parse_formula(specie['pretty_formula'])
	try:
		n_O = elements['O']
	except KeyError:
		n_O = 0.
	return n_O

def get_oxidation_number(specie):
	"""
	Calculates the oxidation number of the metallic element of a binary oxide
	
	Parameters
		specie - dict
			Dictionary of the specie that should contain the key 'unit_cell_formula'
	Returns
		float
			Oxidation number of the metallic element
	"""
	elements = parse_formula(specie['pretty_formula'])
	metal = get_metal('-'.join(elements.keys()))

	try:
		return 2.*float(elements['O'])/float(elements[metal])
	except KeyError:
		return 0.

In [2]:
#Settings related to metal oxides
load_from_matproj = False
save_heat_map = False
matproj_file = 'oxides_data_corrected.json'
properties = ['pretty_formula', 'unit_cell_formula', 'energy', 'e_above_hull']

#Settings related to heat map generation
H_correction = False
verbose = False
T_low = c.T0('K')
T_high = c.convert_unit(num=700., from_='C', to='K')
Ts = np.linspace(T_low, T_high)

P_low = -3
P_high = 3
P_ratios = 10**np.linspace(P_low, P_high)

In [3]:
if load_from_matproj:
	# Message to Saleheen: Generate an API key to Materials Project
	api_key = '3ZvdRbGJX2dwZIaljnX'
	print('Loading data from Materials Project')
	#Get all the binary oxides
	print('Loading oxide species')
	with MPRester(api_key) as m:
		oxides = m.query("*-O", properties = properties)
	#Populate dictionary with stable oxides
	oxide_groups = {}
	for oxide in oxides:
		elements = '-'.join(sorted(oxide['unit_cell_formula'].keys()))
		if np.isclose(oxide['e_above_hull'], 0.):
			try:
				oxide_groups[elements]
			except KeyError:
				oxide_groups[elements] = [oxide]
			else:
				#Check if the oxide has been recorded before
				for prev_oxide in oxide_groups[elements]:
					if oxide['pretty_formula'] == prev_oxide['pretty_formula']:
						break
				else:
					oxide_groups[elements].append(oxide)

	print('Loading metal species')
	#Add metal references
	metal_ref = {}
	for elements, oxide_group in oxide_groups.items():
		metal_element = elements.replace('-O', '').replace('O-', '')
		with MPRester(api_key) as m:
			metals = m.query(metal_element, properties = properties)
		for metal in metals:
			if np.isclose(metal['e_above_hull'], 0.):
				oxide_group.append(metal)
				break
	#Save to json file
	print('Saving to {}'.format(matproj_file))
	with open(matproj_file, 'w') as f_ptr:
		json.dump(oxide_groups, f_ptr)
else:
	print('Loading data from {}'.format(matproj_file))
	with open(matproj_file, 'r') as f_ptr:
		oxide_groups = json.load(f_ptr)

Loading data from oxides_data_corrected.json


In [5]:
#Calculate Gas Phase Molecules Using ASE
H2_thermo = IdealGasThermo(
	vib_energies = c.c('cm/s') * c.h('eV s') * np.array([4306.1793]),
	potentialenergy = -6.7598,
	spin = 0,
	geometry ='linear',
	symmetrynumber = 2,
	atoms = read('H2/CONTCAR')
	)

H2O_thermo = IdealGasThermo(
	vib_energies = c.c('cm/s') * c.h('eV s') * np.array([3825.434, 3710.2642, 1582.432]),
	potentialenergy = -14.2209,
	spin = 0,
	geometry = 'nonlinear',
	symmetrynumber = 2,
	atoms = read('H2O/CONTCAR')
	)

G_H2 = np.array([H2_thermo.get_gibbs_energy(temperature=T, pressure=101325., verbose = False) * c.convert_unit(from_ = 'eV/molecule', to = 'kcal/mol') for T in Ts])
G_H2O = np.array([H2O_thermo.get_gibbs_energy(temperature=T, pressure=101325., verbose = False) * c.convert_unit(from_ = 'eV/molecule', to = 'kcal/mol') for T in Ts])

In [46]:
#Calculate the heat maps given the data
for elements, oxide_group in oxide_groups.items():
	metal = get_metal(elements)
	for (specie1, specie2) in itertools.combinations(oxide_group, 2):
		#Check for the oxidized vs. reduced form
		if get_oxidation_number(specie1) > get_oxidation_number(specie2):
			specie_ox = specie1
			specie_red = specie2
		else:
			specie_ox = specie2
			specie_red = specie1

		n_O_ox = get_oxygen_number(specie_ox)
		n_O_red = get_oxygen_number(specie_red)

		elements_ox = parse_formula(specie_ox['pretty_formula'])
		elements_red = parse_formula(specie_red['pretty_formula'])

		M = np.array([
			[elements_ox[metal], -elements_red[metal]],
			[n_O_ox,             -n_O_red]])
		coefficients = np.linalg.solve(M, np.array([0., 1.]))
		#print('Processing {}{}+H2-->{}{}+H2O'.format(coefficients[0], specie_ox['pretty_formula'], coefficients[1], specie_red['pretty_formula']))

		G_ox = get_unit_energy(specie_ox) * c.convert_unit(from_ = 'eV/molecule', to = 'kcal/mol') 
		G_red = get_unit_energy(specie_red) * c.convert_unit(from_ = 'eV/molecule', to = 'kcal/mol')
		del_G = np.zeros((len(P_ratios), len(Ts)))
		for i, T in enumerate(Ts):
			for j, P_ratio in enumerate(P_ratios):
				del_G[j, i] = coefficients[1] * G_red - coefficients[0] * G_ox + G_H2O[i] - G_H2[i] + np.log(P_ratio) * c.R('kcal/mol/K') * T
				if -0.5 <del_G[j, i] <= 0.5:
					#diff = abs(0.0-del_G[j,i])                 
					#print('{0}\t{1:.2f}\t{2:.2f}\t{3:.2f}\t{4:.2f}'.format(specie_ox['pretty_formula'], specie_red['pretty_formula'], del_G[j,i], T, P_ratio))
					print(f"{specie_ox['pretty_formula']}\t{specie_red['pretty_formula']}\t{del_G[j,i]:.2f}\t{T:.2f}\t{P_ratio:.2f}")

As2O5	As	-0.42	298.15	754.31
As2O5	As	-0.25	298.15	1000.00
As2O5	As	-0.43	311.93	754.31
As2O5	As	-0.26	311.93	1000.00
As2O5	As	-0.45	325.70	754.31
As2O5	As	-0.26	325.70	1000.00
As2O5	As	-0.46	339.48	754.31
As2O5	As	-0.27	339.48	1000.00
As2O5	As	-0.47	353.25	754.31
As2O5	As	-0.27	353.25	1000.00
As2O5	As	-0.49	367.03	754.31
As2O5	As	-0.28	367.03	1000.00
As2O5	As	-0.29	380.80	1000.00
As2O5	As	-0.29	394.58	1000.00
As2O5	As	-0.30	408.35	1000.00
As2O5	As	-0.31	422.13	1000.00
As2O5	As	-0.32	435.91	1000.00
As2O5	As	-0.33	449.68	1000.00
As2O5	As	-0.34	463.46	1000.00
As2O5	As	-0.35	477.23	1000.00
As2O5	As	-0.36	491.01	1000.00
As2O5	As	-0.38	504.78	1000.00
As2O5	As	-0.39	518.56	1000.00
As2O5	As	-0.40	532.33	1000.00
As2O5	As	-0.42	546.11	1000.00
As2O5	As	-0.43	559.88	1000.00
As2O5	As	-0.44	573.66	1000.00
As2O5	As	-0.46	587.44	1000.00
As2O5	As	-0.48	601.21	1000.00
As2O5	As	-0.49	614.99	1000.00
As2O3	As	0.28	422.13	0.00
As2O3	As	-0.10	435.91	0.00
As2O3	As	0.14	435.91	0.00
As2O3	As	0.39	435.91	0.00
A

Co3O4	Co	-0.47	298.15	0.02
Co3O4	Co	-0.30	298.15	0.02
Co3O4	Co	-0.13	298.15	0.03
Co3O4	Co	0.03	298.15	0.04
Co3O4	Co	0.20	298.15	0.05
Co3O4	Co	0.37	298.15	0.07
Co3O4	Co	-0.42	311.93	0.03
Co3O4	Co	-0.25	311.93	0.04
Co3O4	Co	-0.07	311.93	0.05
Co3O4	Co	0.10	311.93	0.07
Co3O4	Co	0.28	311.93	0.09
Co3O4	Co	0.45	311.93	0.12
Co3O4	Co	-0.35	325.70	0.05
Co3O4	Co	-0.16	325.70	0.07
Co3O4	Co	0.02	325.70	0.09
Co3O4	Co	0.20	325.70	0.12
Co3O4	Co	0.38	325.70	0.16
Co3O4	Co	-0.43	339.48	0.07
Co3O4	Co	-0.24	339.48	0.09
Co3O4	Co	-0.05	339.48	0.12
Co3O4	Co	0.14	339.48	0.16
Co3O4	Co	0.33	339.48	0.21
Co3O4	Co	-0.30	353.25	0.12
Co3O4	Co	-0.11	353.25	0.16
Co3O4	Co	0.09	353.25	0.21
Co3O4	Co	0.29	353.25	0.28
Co3O4	Co	0.49	353.25	0.37
Co3O4	Co	-0.35	367.03	0.16
Co3O4	Co	-0.15	367.03	0.21
Co3O4	Co	0.06	367.03	0.28
Co3O4	Co	0.26	367.03	0.37
Co3O4	Co	0.47	367.03	0.49
Co3O4	Co	-0.39	380.80	0.21
Co3O4	Co	-0.17	380.80	0.28
Co3O4	Co	0.04	380.80	0.37
Co3O4	Co	0.26	380.80	0.49
Co3O4	Co	0.47	380.80	0.66
Co3O4	Co	-0.40	394.58

Cs2O	Cs11O3	0.38	973.15	0.00
Cs2O	Cs3O	0.49	862.95	0.00
Cs2O	Cs3O	0.09	876.72	0.00
Cs2O	Cs3O	-0.32	890.50	0.00
Cs2O	Cs3O	0.18	890.50	0.00
Cs2O	Cs3O	-0.22	904.27	0.00
Cs2O	Cs3O	0.29	904.27	0.00
Cs2O	Cs3O	-0.10	918.05	0.00
Cs2O	Cs3O	0.41	918.05	0.00
Cs2O	Cs3O	-0.49	931.82	0.00
Cs2O	Cs3O	0.03	931.82	0.00
Cs2O	Cs3O	-0.35	945.60	0.00
Cs2O	Cs3O	0.18	945.60	0.00
Cs2O	Cs3O	-0.20	959.37	0.00
Cs2O	Cs3O	0.34	959.37	0.00
Cs2O	Cs3O	-0.03	973.15	0.00
Cs2O2	Cs	-0.39	298.15	0.04
Cs2O2	Cs	-0.22	298.15	0.05
Cs2O2	Cs	-0.06	298.15	0.07
Cs2O2	Cs	0.11	298.15	0.09
Cs2O2	Cs	0.28	298.15	0.12
Cs2O2	Cs	0.45	298.15	0.16
Cs2O2	Cs	-0.50	311.93	0.05
Cs2O2	Cs	-0.32	311.93	0.07
Cs2O2	Cs	-0.15	311.93	0.09
Cs2O2	Cs	0.03	311.93	0.12
Cs2O2	Cs	0.20	311.93	0.16
Cs2O2	Cs	0.38	311.93	0.21
Cs2O2	Cs	-0.41	325.70	0.09
Cs2O2	Cs	-0.22	325.70	0.12
Cs2O2	Cs	-0.04	325.70	0.16
Cs2O2	Cs	0.14	325.70	0.21
Cs2O2	Cs	0.32	325.70	0.28
Cs2O2	Cs	-0.48	339.48	0.12
Cs2O2	Cs	-0.29	339.48	0.16
Cs2O2	Cs	-0.10	339.48	0.21
Cs2O2	Cs	0.09	339.48	0.28
C

GeO2	Ge	0.19	752.74	0.00
GeO2	Ge	-0.21	766.52	0.00
GeO2	Ge	0.22	766.52	0.00
GeO2	Ge	-0.17	780.29	0.00
GeO2	Ge	0.27	780.29	0.00
GeO2	Ge	-0.12	794.07	0.00
GeO2	Ge	0.33	794.07	0.00
GeO2	Ge	-0.05	807.84	0.00
GeO2	Ge	0.40	807.84	0.00
GeO2	Ge	-0.43	821.62	0.00
GeO2	Ge	0.03	821.62	0.00
GeO2	Ge	0.49	821.62	0.00
GeO2	Ge	-0.34	835.39	0.00
GeO2	Ge	0.12	835.39	0.00
GeO2	Ge	-0.24	849.17	0.00
GeO2	Ge	0.23	849.17	0.01
GeO2	Ge	-0.12	862.95	0.01
GeO2	Ge	0.36	862.95	0.01
GeO2	Ge	-0.48	876.72	0.01
GeO2	Ge	0.01	876.72	0.01
GeO2	Ge	-0.34	890.50	0.01
GeO2	Ge	0.16	890.50	0.01
GeO2	Ge	-0.19	904.27	0.01
GeO2	Ge	0.32	904.27	0.01
GeO2	Ge	-0.02	918.05	0.01
GeO2	Ge	0.50	918.05	0.02
GeO2	Ge	-0.35	931.82	0.01
GeO2	Ge	0.17	931.82	0.02
GeO2	Ge	-0.16	945.60	0.02
GeO2	Ge	0.37	945.60	0.02
GeO2	Ge	-0.49	959.37	0.02
GeO2	Ge	0.04	959.37	0.02
GeO2	Ge	-0.28	973.15	0.02
GeO2	Ge	0.27	973.15	0.03
In2O3	In	0.31	918.05	0.00
In2O3	In	-0.10	931.82	0.00
In2O3	In	0.42	931.82	0.00
In2O3	In	0.02	945.60	0.00
In2O3	In	-0.38	959.37	0.00
In

Mn3O4	MnO	-0.13	780.29	44.98
Mn3O4	MnO	0.31	780.29	59.64
Mn3O4	MnO	-0.24	794.07	44.98
Mn3O4	MnO	0.21	794.07	59.64
Mn3O4	MnO	-0.35	807.84	44.98
Mn3O4	MnO	0.11	807.84	59.64
Mn3O4	MnO	-0.45	821.62	44.98
Mn3O4	MnO	0.01	821.62	59.64
Mn3O4	MnO	0.47	821.62	79.06
Mn3O4	MnO	-0.10	835.39	59.64
Mn3O4	MnO	0.37	835.39	79.06
Mn3O4	MnO	-0.20	849.17	59.64
Mn3O4	MnO	0.28	849.17	79.06
Mn3O4	MnO	-0.30	862.95	59.64
Mn3O4	MnO	0.18	862.95	79.06
Mn3O4	MnO	-0.41	876.72	59.64
Mn3O4	MnO	0.09	876.72	79.06
Mn3O4	MnO	-0.01	890.50	79.06
Mn3O4	MnO	0.49	890.50	104.81
Mn3O4	MnO	-0.11	904.27	79.06
Mn3O4	MnO	0.40	904.27	104.81
Mn3O4	MnO	-0.21	918.05	79.06
Mn3O4	MnO	0.31	918.05	104.81
Mn3O4	MnO	-0.30	931.82	79.06
Mn3O4	MnO	0.22	931.82	104.81
Mn3O4	MnO	-0.40	945.60	79.06
Mn3O4	MnO	0.13	945.60	104.81
Mn3O4	MnO	0.04	959.37	104.81
Mn3O4	MnO	-0.05	973.15	104.81
Mn3O4	MnO	0.49	973.15	138.95
MnO2	Mn	0.22	408.35	0.00
MnO2	Mn	0.45	408.35	0.00
MnO2	Mn	-0.17	422.13	0.00
MnO2	Mn	0.07	422.13	0.00
MnO2	Mn	0.31	422.13	0.00
MnO2	Mn	-0.3

...

Sn5O6	SnO	0.16	725.19	0.00
Sn5O6	SnO	-0.24	738.97	0.00
Sn5O6	SnO	0.17	738.97	0.00
Sn5O6	SnO	-0.22	752.74	0.00
Sn5O6	SnO	0.20	752.74	0.00
Sn5O6	SnO	-0.18	766.52	0.00
Sn5O6	SnO	0.24	766.52	0.00
Sn5O6	SnO	-0.13	780.29	0.00
Sn5O6	SnO	0.30	780.29	0.00
Sn5O6	SnO	-0.07	794.07	0.00
Sn5O6	SnO	0.38	794.07	0.00
Sn5O6	SnO	-0.44	807.84	0.00
Sn5O6	SnO	0.02	807.84	0.00
Sn5O6	SnO	0.47	807.84	0.01
Sn5O6	SnO	-0.35	821.62	0.00
Sn5O6	SnO	0.11	821.62	0.01
Sn5O6	SnO	-0.25	835.39	0.01
Sn5O6	SnO	0.22	835.39	0.01
Sn5O6	SnO	-0.13	849.17	0.01
Sn5O6	SnO	0.35	849.17	0.01
Sn5O6	SnO	-0.48	862.95	0.01
Sn5O6	SnO	0.01	862.95	0.01
Sn5O6	SnO	0.49	862.95	0.01
Sn5O6	SnO	-0.34	876.72	0.01
Sn5O6	SnO	0.16	876.72	0.01
Sn5O6	SnO	-0.18	890.50	0.01
Sn5O6	SnO	0.32	890.50	0.02
Sn5O6	SnO	-0.01	904.27	0.02
Sn5O6	SnO	0.50	904.27	0.02
Sn5O6	SnO	-0.34	918.05	0.02
Sn5O6	SnO	0.18	918.05	0.02
Sn5O6	SnO	-0.15	931.82	0.02
Sn5O6	SnO	0.38	931.82	0.03
Sn5O6	SnO	-0.47	945.60	0.02
Sn5O6	SnO	0.06	945.60	0.03
Sn5O6	SnO	-0.25	959.37	0.03
Sn5O6	SnO	0

TcO2	Tc	0.21	518.56	0.00
TcO2	Tc	0.50	518.56	0.00
TcO2	Tc	-0.18	532.33	0.00
TcO2	Tc	0.11	532.33	0.00
TcO2	Tc	0.41	532.33	0.00
TcO2	Tc	-0.27	546.11	0.00
TcO2	Tc	0.04	546.11	0.00
TcO2	Tc	0.34	546.11	0.00
TcO2	Tc	-0.34	559.88	0.00
TcO2	Tc	-0.03	559.88	0.00
TcO2	Tc	0.29	559.88	0.00
TcO2	Tc	-0.40	573.66	0.00
TcO2	Tc	-0.08	573.66	0.00
TcO2	Tc	0.25	573.66	0.00
TcO2	Tc	-0.44	587.44	0.00
TcO2	Tc	-0.11	587.44	0.00
TcO2	Tc	0.22	587.44	0.01
TcO2	Tc	-0.46	601.21	0.00
TcO2	Tc	-0.13	601.21	0.01
TcO2	Tc	0.21	601.21	0.01
TcO2	Tc	-0.48	614.99	0.01
TcO2	Tc	-0.13	614.99	0.01
TcO2	Tc	0.21	614.99	0.01
TcO2	Tc	-0.47	628.76	0.01
TcO2	Tc	-0.12	628.76	0.01
TcO2	Tc	0.23	628.76	0.01
TcO2	Tc	-0.45	642.54	0.01
TcO2	Tc	-0.09	642.54	0.01
TcO2	Tc	0.27	642.54	0.02
TcO2	Tc	-0.42	656.31	0.01
TcO2	Tc	-0.05	656.31	0.02
TcO2	Tc	0.31	656.31	0.02
TcO2	Tc	-0.37	670.09	0.02
TcO2	Tc	0.00	670.09	0.02
TcO2	Tc	0.38	670.09	0.03
TcO2	Tc	-0.31	683.86	0.02
TcO2	Tc	0.07	683.86	0.03
TcO2	Tc	0.46	683.86	0.04
TcO2	Tc	-0.23	697.64	0.03
TcO2

V3O5	V2O3	-0.37	298.15	1.15
V3O5	V2O3	-0.20	298.15	1.53
V3O5	V2O3	-0.03	298.15	2.02
V3O5	V2O3	0.13	298.15	2.68
V3O5	V2O3	0.30	298.15	3.56
V3O5	V2O3	0.47	298.15	4.71
V3O5	V2O3	-0.38	311.93	1.53
V3O5	V2O3	-0.21	311.93	2.02
V3O5	V2O3	-0.03	311.93	2.68
V3O5	V2O3	0.14	311.93	3.56
V3O5	V2O3	0.32	311.93	4.71
V3O5	V2O3	0.49	311.93	6.25
V3O5	V2O3	-0.38	325.70	2.02
V3O5	V2O3	-0.20	325.70	2.68
V3O5	V2O3	-0.02	325.70	3.56
V3O5	V2O3	0.17	325.70	4.71
V3O5	V2O3	0.35	325.70	6.25
V3O5	V2O3	-0.37	339.48	2.68
V3O5	V2O3	-0.18	339.48	3.56
V3O5	V2O3	0.01	339.48	4.71
V3O5	V2O3	0.21	339.48	6.25
V3O5	V2O3	0.40	339.48	8.29
V3O5	V2O3	-0.34	353.25	3.56
V3O5	V2O3	-0.14	353.25	4.71
V3O5	V2O3	0.06	353.25	6.25
V3O5	V2O3	0.26	353.25	8.29
V3O5	V2O3	0.46	353.25	10.99
V3O5	V2O3	-0.50	367.03	3.56
V3O5	V2O3	-0.29	367.03	4.71
V3O5	V2O3	-0.08	367.03	6.25
V3O5	V2O3	0.12	367.03	8.29
V3O5	V2O3	0.33	367.03	10.99
V3O5	V2O3	-0.44	380.80	4.71
V3O5	V2O3	-0.23	380.80	6.25
V3O5	V2O3	-0.02	380.80	8.29
V3O5	V2O3	0.20	380.80	10.99
V3O5	V

V2O5	V	0.45	807.84	0.00
V2O5	V	0.05	821.62	0.00
V2O5	V	-0.36	835.39	0.00
V2O5	V	0.11	835.39	0.00
V2O5	V	-0.28	849.17	0.00
V2O5	V	0.19	849.17	0.00
V2O5	V	-0.20	862.95	0.00
V2O5	V	0.29	862.95	0.00
V2O5	V	-0.10	876.72	0.00
V2O5	V	0.40	876.72	0.00
V2O5	V	-0.48	890.50	0.00
V2O5	V	0.02	890.50	0.00
V2O5	V	-0.35	904.27	0.00
V2O5	V	0.15	904.27	0.00
V2O5	V	-0.21	918.05	0.00
V2O5	V	0.30	918.05	0.01
V2O5	V	-0.06	931.82	0.01
V2O5	V	0.46	931.82	0.01
V2O5	V	-0.42	945.60	0.01
V2O5	V	0.11	945.60	0.01
V2O5	V	-0.25	959.37	0.01
V2O5	V	0.29	959.37	0.01
V2O5	V	-0.05	973.15	0.01
V2O5	V	0.49	973.15	0.01
W18O49	WO2	0.20	353.25	0.00
W18O49	WO2	0.40	353.25	0.00
W18O49	WO2	-0.18	367.03	0.00
W18O49	WO2	0.02	367.03	0.00
W18O49	WO2	0.23	367.03	0.00
W18O49	WO2	0.43	367.03	0.00
W18O49	WO2	-0.36	380.80	0.00
W18O49	WO2	-0.14	380.80	0.00
W18O49	WO2	0.07	380.80	0.00
W18O49	WO2	0.28	380.80	0.00
W18O49	WO2	0.50	380.80	0.00
W18O49	WO2	-0.29	394.58	0.00
W18O49	WO2	-0.07	394.58	0.00
W18O49	WO2	0.15	394.58	0.00
W18O49	WO2	0.37	

WO3	W	0.06	573.66	0.00
WO3	W	0.39	573.66	0.00
WO3	W	-0.33	587.44	0.00
WO3	W	-0.00	587.44	0.00
WO3	W	0.33	587.44	0.00
WO3	W	-0.39	601.21	0.00
WO3	W	-0.05	601.21	0.00
WO3	W	0.29	601.21	0.00
WO3	W	-0.43	614.99	0.00
WO3	W	-0.08	614.99	0.00
WO3	W	0.26	614.99	0.00
WO3	W	-0.46	628.76	0.00
WO3	W	-0.10	628.76	0.00
WO3	W	0.25	628.76	0.00
WO3	W	-0.47	642.54	0.00
WO3	W	-0.11	642.54	0.00
WO3	W	0.25	642.54	0.01
WO3	W	-0.47	656.31	0.00
WO3	W	-0.10	656.31	0.01
WO3	W	0.27	656.31	0.01
WO3	W	-0.45	670.09	0.01
WO3	W	-0.07	670.09	0.01
WO3	W	0.30	670.09	0.01
WO3	W	-0.42	683.86	0.01
WO3	W	-0.03	683.86	0.01
WO3	W	0.35	683.86	0.01
WO3	W	-0.37	697.64	0.01
WO3	W	0.02	697.64	0.01
WO3	W	0.41	697.64	0.02
WO3	W	-0.31	711.42	0.01
WO3	W	0.09	711.42	0.02
WO3	W	0.49	711.42	0.02
WO3	W	-0.23	725.19	0.02
WO3	W	0.18	725.19	0.02
WO3	W	-0.14	738.97	0.02
WO3	W	0.27	738.97	0.03
WO3	W	-0.45	752.74	0.02
WO3	W	-0.03	752.74	0.03
WO3	W	0.39	752.74	0.04
WO3	W	-0.34	766.52	0.03
WO3	W	0.09	766.52	0.04
WO3	W	-0.21	780.29	0.04
WO3	W	0.23